In [2]:
# notebooks/01_data_collection_preprocessing.ipynb

import os
import sys
import pandas as pd
from google_play_scraper import Sort, reviews_all
import tqdm # For progress bar in notebooks


In [3]:

# --- Project Setup: Ensure src module is discoverable ---
# This block is crucial for importing from src.config
def find_project_root(current_path):
    """
    Finds the project root by looking for common project directories.
    Assumes project_root contains 'src', 'data', and 'notebooks'.
    """
    path = current_path
    while path != os.path.dirname(path):
        if (os.path.isdir(os.path.join(path, 'src')) and
            os.path.isdir(os.path.join(path, 'data')) and
            os.path.isdir(os.path.join(path, 'notebooks'))):
            return path
        path = os.path.dirname(path)
    return current_path # Fallback if no specific root found

current_working_dir = os.getcwd()
project_root = find_project_root(current_working_dir)

if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added '{project_root}' to sys.path for module imports.")
else:
    print(f"'{project_root}' already in sys.path.")

# Import configuration variables from src.config
from src.config import APP_IDS, MIN_REVIEWS_PER_APP, RAW_REVIEWS_CSV, CLEAN_REVIEWS_CSV

Added 'c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-2\fintech-app-customer-experience-analytics' to sys.path for module imports.
Project structure setup complete and config.py created/updated.
Base Directory: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-2\fintech-app-customer-experience-analytics
Raw Data Directory: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-2\fintech-app-customer-experience-analytics\data\raw
Processed Data Directory: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-2\fintech-app-customer-experience-analytics\data\processed
App IDs to scrape: {'Commercial Bank of Ethiopia': 'com.cbe.mobile.banking', 'Bank of Abyssinia': 'com.boa.mobile.banking', 'Dashen Bank': 'com.dashen.mobile.banking'}


In [ ]:


print("\n--- Starting Data Collection and Preprocessing ---")

# --- Step 1: Web Scraping ---
print("\n--- Initiating Google Play Store Review Scraping ---")

all_reviews = []
total_reviews_collected = 0

# Loop through each app defined in config.py
for bank_name, app_id in APP_IDS.items():
    print(f"\nScraping reviews for {bank_name} (App ID: {app_id})...")
    
    try:
        # Use reviews_all to get as many reviews as possible up to a limit
        # reviews_all fetches all reviews and handles pagination
        app_reviews = reviews_all(
            app_id,
            lang='en', # Language of reviews
            country='et', # Country for reviews (Ethiopia)
            sort=Sort.NEWEST, # Sort by newest reviews
            count=MIN_REVIEWS_PER_APP + 50 # Try to fetch a few more than minimum, just in case
        )
        
        # Process collected reviews
        for review in app_reviews:
            all_reviews.append({
                'review_text': review['content'],
                'rating': review['score'],
                'date': review['at'],
                'bank_name': bank_name,
                'source': 'Google Play'
            })
        
        collected_count = len(app_reviews)
        total_reviews_collected += collected_count
        print(f"Collected {collected_count} reviews for {bank_name}.")
        
    except Exception as e:
        print(f"ERROR: Could not scrape reviews for {bank_name} (App ID: {app_id}). Error: {e}")
        # If scraping fails, continue to the next app rather than stopping the whole process

if not all_reviews:
    print("CRITICAL WARNING: No reviews were collected for any app. Please check App IDs and internet connection.")
    # sys.exit("Exiting: No reviews collected.") # Don't exit here, allow processing to continue with empty df


# Convert collected reviews to a DataFrame
raw_reviews_df = pd.DataFrame(all_reviews)

if raw_reviews_df.empty:
    print("\nNo reviews collected. Raw DataFrame is empty.")
else:
    print(f"\n--- Raw Reviews Collection Summary ---")
    print(f"Total raw reviews collected: {len(raw_reviews_df)}")
    print("Raw DataFrame Info:")
    raw_reviews_df.info()
    print("\nFirst 5 rows of Raw Reviews:")
    print(raw_reviews_df.head())
    print(f"Saving raw reviews to: {RAW_REVIEWS_CSV}")
    raw_reviews_df.to_csv(RAW_REVIEWS_CSV, index=False)


In [ ]:


# --- Step 2: Preprocessing ---
print("\n--- Initiating Review Preprocessing ---")

if raw_reviews_df.empty:
    print("Raw reviews DataFrame is empty. Skipping preprocessing.")
    clean_reviews_df = pd.DataFrame(columns=['review_text', 'rating', 'date', 'bank_name', 'source']) # Create empty structure
else:
    clean_reviews_df = raw_reviews_df.copy()

    # Remove duplicates based on review_id
    initial_count = len(clean_reviews_df)
    clean_reviews_df.drop_duplicates(subset=['review_id'], inplace=True)
    if len(clean_reviews_df) < initial_count:
        print(f"Removed {initial_count - len(clean_reviews_df)} duplicate reviews.")

    # Handle missing data: Drop rows where review_text or rating are missing as they are critical
    initial_count = len(clean_reviews_df)
    clean_reviews_df.dropna(subset=['review_text', 'rating'], inplace=True)
    if len(clean_reviews_df) < initial_count:
        print(f"Removed {initial_count - len(clean_reviews_df)} rows with missing critical data (review_text/rating).")
    
    # Normalize dates: Convert 'date' column to YYYY-MM-DD format
    # The 'at' field from google-play-scraper returns datetime objects directly,
    # so we just need to convert to date only.
    clean_reviews_df['date'] = pd.to_datetime(clean_reviews_df['date'], errors='coerce').dt.date
    
    # Drop rows where date conversion might have failed (if any)
    initial_count = len(clean_reviews_df)
    clean_reviews_df.dropna(subset=['date'], inplace=True)
    if len(clean_reviews_df) < initial_count:
        print(f"Removed {initial_count - len(clean_reviews_df)} rows with unparseable dates.")

    # Select and reorder columns as specified in deliverables
    clean_reviews_df = clean_reviews_df[['review_id', 'review_text', 'rating', 'date', 'bank_name', 'source']]

    # Check for KPI: 1,200+ reviews collected with <5% missing data
    total_reviews_final = len(clean_reviews_df)
    missing_data_percentage = (clean_reviews_df.isnull().sum().sum() / clean_reviews_df.size) * 100
    
    print(f"\n--- Preprocessing Summary ---")
    print(f"Total clean reviews after preprocessing: {total_reviews_final}")
    print(f"Missing data percentage in clean DataFrame: {missing_data_percentage:.2f}%")

    if total_reviews_final >= (len(APP_IDS) * MIN_REVIEWS_PER_APP) and missing_data_percentage < 5:
        print("KPI Met: Successfully collected required reviews with low missing data.")
    elif total_reviews_final < (len(APP_IDS) * MIN_REVIEWS_PER_APP):
        print(f"KPI Warning: Did not meet target of {len(APP_IDS) * MIN_REVIEWS_PER_APP} reviews. Collected {total_reviews_final}.")
    elif missing_data_percentage >= 5:
        print(f"KPI Warning: Missing data percentage ({missing_data_percentage:.2f}%) is 5% or higher.")
    
    print("\nClean DataFrame Info:")
    clean_reviews_df.info()
    print("\nFirst 5 rows of Clean Reviews:")
    print(clean_reviews_df.head())

    print(f"Saving clean reviews to: {CLEAN_REVIEWS_CSV}")
    clean_reviews_df.to_csv(CLEAN_REVIEWS_CSV, index=False)

print("\n--- Task 1: Data Collection and Preprocessing Complete ---")
print("Remember to commit your work to your 'task-1' branch!")
